A Gentle Introduction to Large Language Models – materials
==========================================================

**Date:** [2023-07-17 Mon 11:48]



## Trajectory of the words generating process



In [1]:
using OpenAI
secret_key = "your secret key"
ENV["HTTP_PROXY"] = "your proxy" 
ENV["HTTPS_PROXY"] = "your proxy"

### Single trajectory



-   Generate an entire paragraph



In [1]:
response_single_sentence = create_chat(
    secret_key,
    "gpt-3.5-turbo",
    [Dict("role"=>"system", "content"=>"Please finish the sentence in 500 words"),
     Dict("role"=>"user", "content"=>"Physics is")],
    n = 1,
    # logprobs = 5,
    max_tokens = 4000,
    temperature = 1
);

In [1]:
print(response_single_sentence.response.choices[1]["message"]["content"]);

#+begin_example
Physics is the scientific study of matter, energy, and the interaction between them. It is a branch of science that aims to understand the fundamental laws of nature and describe how the physical world functions.

At its core, physics seeks to explain the behavior of objects and phenomena in the universe using mathematical equations and theoretical frameworks. It deals with concepts such as motion, force, energy, and the structure of matter. Moreover, it explores the behavior of matter and energy at the atomic and subatomic levels, delving into the realm of quantum mechanics and particle physics.

One of the key principles in physics is the concept of cause and effect. It provides a framework for understanding and predicting how objects and systems will respond to various stimuli. For example, Newton's laws of motion describe how an object will move in response to a force acting upon it. These laws are foundational to many aspects of physics, from the study of celestial

-   For each generated word, get the embedding vector



In [1]:
splitted_text = split(response_single_sentence.response.choices[1]["message"]["content"])
reconstruction = ["Physics is"]

for word in splitted_text
    push!(reconstruction, string(reconstruction[end]," ", word))
end

In [1]:
reconstruction

In [1]:
# reconstruction = ["Physics", "Physics is ", "Physics is a", "Physics is a scientific"]
embeddings_single_sentence = create_embeddings(secret_key,
                                               reconstruction);

embeddings_single_sentence_array = []
for embedding in embeddings_single_sentence.response["data"]
    push!(embeddings_single_sentence_array,embedding["embedding"])
end

In [1]:
embeddings_single_sentence_array

In [1]:
using MultivariateStats
using Plots
using TSne
matrix_embeddings = hcat(embeddings_single_sentence_array...);

In [1]:
# Perform t-SNE
# Y = tsne(matrix_embeddings', 2, 0, 5000, ) # Y is now a 2x500 matrix
M = fit(PCA, matrix_embeddings; maxoutdim=2)
Y2 = transform(M, matrix_embeddings)

2×501 Matrix{Float64}:
 0.227534   0.238293   0.258116  …  -0.0577011  -0.0574144  -0.0616565
 0.0994556  0.0998893  0.135478      0.0615511   0.0635793   0.0599092

In [1]:
# Create scatter plot
plot(Y2[1,:], Y2[2,:], legend=false, line_z=1:length(Y2), colorbar=true)
title!("Trajectory of 'Physics is ...'")
xlabel!("x-dimension")
ylabel!("y-dimension")

#+attr_org: :width 1143

### Multiple trajectory



#### Generate multiple paragraph



In [1]:
response_multiple_paragraph = create_chat(
    secret_key,
    "gpt-3.5-turbo",
    [Dict("role"=>"system", "content"=>"Please finish the sentence in 105 words"),
     Dict("role"=>"user", "content"=>"Physics is")],
    n = 19,
    # logprobs = 5,
    max_tokens = 4000,
    temperature = 1
);

#### For each paragraph get the incremental embedding vector



In [1]:
reconstruction_all = []
for response in response_multiple_paragraph.response.choices
    reconstruction = ["Physics is"]
    splitted_text = split(response["message"]["content"])
    for word in splitted_text
        push!(reconstruction, string(reconstruction[end]," ", word))
    end
    push!(reconstruction_all, reconstruction)
end
reconstruction_flattened = reduce(vcat, reconstruction_all);

In [1]:
length(reconstruction_flattened)

2005

In [1]:
length_paragraphs = pushfirst!(cumsum(length.(reconstruction_all)),1)

#### Reduce dimension to 2



In [1]:
using MultivariateStats
using Random
using Plots
using TSne
# ProgressMeter.ijulia_behavior(:clear)
Random.seed!(42)
# embedding_2d_all = []
# reconstruction_all_fake = [["Physics"],["Physics"],["Physics"],["Physics"],["Physics"]]

embeddings_paragraphs_all = create_embeddings(secret_key,
                                         reconstruction_flattened);

In [1]:
embeddings_paragraph = [embeddings["embedding"] for embeddings in embeddings_paragraphs_all.response["data"]];

In [1]:
matrix_embeddings = hcat(embeddings_paragraph...);

In [1]:
M = fit(PCA, matrix_embeddings, maxoutdim=2)
Y = transform(M, matrix_embeddings)

2×2005 Matrix{Float64}:
 -0.162331  -0.218734   -0.182849   …   0.0996091   0.0898563   0.0957893
 -0.059319  -0.0680076  -0.0446089     -0.0578989  -0.0563043  -0.0621402

In [1]:
# for ind in 1:length(reconstruction_all)
#     embeddings_paragraph_array = []
#     embeddings_paragraph = embeddings_paragraphs_all.response["data"][length_paragraphs[ind]:length_paragraphs[ind+1]-1]
#     for embedding in embeddings_paragraph
#         push!(embeddings_paragraph_array, embedding["embedding"])
#     end

#     # print(matrix_embeddings[1:2])
#     # print(matrix_embeddings[:,1])
#     # Y = tsne(matrix_embeddings', 2, 0, 1000);
#     push!(embedding_2d_all, Y);
# end

In [1]:
# embeddings_paragraph_array = []
# embeddings_paragraph = embeddings_paragraphs_all.response["data"][length_paragraphs[1]:length_paragraphs[2]]
# for embedding in embeddings_paragraph
#     push!(embeddings_paragraph_array, embedding["embedding"])
# end

In [1]:
# embeddings_paragraph_array[3]

In [1]:
p = plot(Y[1,length_paragraphs[1]:length_paragraphs[2]-1], Y[2,length_paragraphs[1]:length_paragraphs[2]-1], legend=false, line_z=1:length(length_paragraphs[1]:length_paragraphs[2]-1), colorbar=true)
# plot(p, Y[1,length_paragraphs[2]:length_paragraphs[3]-1], Y[2,length_paragraphs[2]:length_paragraphs[3]-1], legend=false, line_z=1:length(length_paragraphs[2]:length_paragraphs[3]-1), colorbar=true)
# plot(p, Y[1,length_paragraphs[3]:length_paragraphs[4]-1], Y[2,length_paragraphs[3]:length_paragraphs[4]-1], legend=false, line_z=1:length(length_paragraphs[3]:length_paragraphs[4]-1), colorbar=true)
for i in 2:length(length_paragraphs) - 1
    plot!(p,Y[1,length_paragraphs[i]+1:length_paragraphs[i+1] - 1], Y[2,length_paragraphs[i]+1:length_paragraphs[i+1] - 1], legend=false, line_z=1:length(length_paragraphs[i]+1:length_paragraphs[i+1]-1), colorbar=true)
    scatter!(p,[Y[1,length_paragraphs[i]+1]], [Y[2,length_paragraphs[i]+1]])
end
title!("Trajectory of 'Physics is ...'")
xlabel!("x-dimension")
ylabel!("y-dimension")

#+attr_org: :width 1143

In [1]:
p = plot(Y[1,length_paragraphs[1]:length_paragraphs[2]-1], Y[2,length_paragraphs[1]:length_paragraphs[2]-1], legend=true, label="Trajectory 1")

for i in 2:length(length_paragraphs) - 1
    plot!(p,Y[1,length_paragraphs[i]+1:length_paragraphs[i+1] - 1], Y[2,length_paragraphs[i]+1:length_paragraphs[i+1] - 1], legend=true, label="Trajectory $(i)")
    scatter!(p,[Y[1,length_paragraphs[i]+1]], [Y[2,length_paragraphs[i]+1]],label="")
end
title!("Trajectory of 'Physics is ...'")
xlabel!("x-dimension")
ylabel!("y-dimension")

#+attr_org: :width 1146

In [1]:
print(response_multiple_paragraph.response.choices[1]["message"]["content"])

the scientific study of matter, energy, and the interactions between the two. It aims to understand the fundamental laws and principles that govern the universe, from the smallest particles to the vast expanse of space. Physics seeks to explain the natural phenomena we observe and predict how they will behave in the future. It is a discipline that combines theoretical models, mathematical equations, and experimental observations to uncover the secrets of nature. From classical mechanics to quantum physics, from relativity to the study of the fundamental forces, physics is a constantly evolving field that has revolutionized our understanding of the world we live in.

In [1]:
print(response_multiple_paragraph.response.choices[15]["message"]["content"])

a branch of science that deals with the study of matter, energy, and the interactions between them. It seeks to understand the fundamental laws and principles that govern the natural world, from the behavior of subatomic particles to the motion of astronomical bodies. Physics encompasses a wide range of topics, including classical mechanics, thermodynamics, electromagnetism, quantum mechanics, and relativity. It involves both theoretical and experimental approaches, with scientists using mathematical models and sophisticated instruments to observe and analyze physical phenomena. The knowledge gained from studying physics has led to numerous technological advancements and has deepened our understanding of the universe and our place within it.

## Be clear and specific



In [1]:
response_clear_specific_paragraph = create_chat(
    secret_key,
    "gpt-3.5-turbo",
    [Dict("role"=>"system", "content"=>"Please finish the sentence in 105 words and follow the instructions:\n
    Clearly define physics by the object it studies in 15 words.\n
    List the subfields physics in 45 words.\n
    Elaborate on how physics affect the everyday life of human beings in 45 words\n"),
     Dict("role"=>"user", "content"=>"Physics is")],
    n = 19,
    # logprobs = 5,
    max_tokens = 4000,
    temperature = 1
);

In [1]:
reconstruction_clear_specific_all = []
reconstruction_clear_specific_flattened = []
for response in response_clear_specific_paragraph.response.choices
    reconstruction_clear_specific = ["Physics is"]
    splitted_text = split(response["message"]["content"])
    for word in splitted_text
        push!(reconstruction_clear_specific, string(reconstruction_clear_specific[end]," ", word))
    end
    push!(reconstruction_clear_specific_all, reconstruction_clear_specific)
end
reconstruction_clear_specific_flattened = reduce(vcat, reconstruction_clear_specific_all);

In [1]:
# response_clear_specific_paragraph.response.choices[1]["message"]["content"]
# splitted_text
# reconstruction_clear_specific
reconstruction_clear_specific_flattened[18]

"Physics is the study of matter and energy and the interactions between them. Subfields of physics include classical mechanics,"

In [1]:
length_paragraphs_cs = pushfirst!(cumsum(length.(reconstruction_clear_specific_all)),1)
length(reconstruction_clear_specific_flattened)

1694

In [1]:
print(reconstruction_clear_specific_all[1][30])

Physics is the study of matter, energy, and the interactions between them. The subfields of physics include mechanics, thermodynamics, electromagnetism, optics, quantum mechanics, and particle physics. Physics affects everyday life in

### Reduce dimension to 2



In [1]:
using MultivariateStats
using Random
using Plots
embeddings_paragraphs_clear_specific_all = create_embeddings(secret_key,
                                                             reconstruction_clear_specific_flattened);

In [1]:
embeddings_paragraph_clear_specific = [embeddings["embedding"] for embeddings in embeddings_paragraphs_clear_specific_all.response["data"]];
matrix_embeddings_clear_specific = hcat(embeddings_paragraph_clear_specific...);
M_cs = fit(PCA, matrix_embeddings_clear_specific, maxoutdim=2)
Y_cs = transform(M_cs, matrix_embeddings_clear_specific);

### Plot the results



In [1]:
p_cs = plot(Y_cs[1,length_paragraphs_cs[1]:length_paragraphs_cs[2]-1], Y_cs[2,length_paragraphs_cs[1]:length_paragraphs_cs[2]-1], legend=false, line_z=1:length(length_paragraphs_cs[1]:length_paragraphs_cs[2]-1), colorbar=true)
# plot(p, Y_cs[1,length_paragraphs_cs[2]:length_paragraphs_cs[3]-1], Y_cs[2,length_paragraphs_cs[2]:length_paragraphs_cs[3]-1], legend=false, line_z=1:length(length_paragraphs_cs[2]:length_paragraphs_cs[3]-1), colorbar=true)
# plot(p, Y_cs[1,length_paragraphs_cs[3]:length_paragraphs_cs[4]-1], Y_cs[2,length_paragraphs_cs[3]:length_paragraphs_cs[4]-1], legend=false, line_z=1:length(length_paragraphs_cs[3]:length_paragraphs_cs[4]-1), colorbar=true)
for i in 2:length(length_paragraphs_cs) - 1
    plot!(p_cs,Y_cs[1,length_paragraphs_cs[i]+1:length_paragraphs_cs[i+1] - 1], Y_cs[2,length_paragraphs_cs[i]+1:length_paragraphs_cs[i+1] - 1], legend=false, line_z=1:length(length_paragraphs_cs[i]+1:length_paragraphs_cs[i+1]-1), colorbar=true)
    scatter!(p_cs,[Y_cs[1,length_paragraphs_cs[i]+1]], [Y_cs[2,length_paragraphs_cs[i]+1]])
end
title!("Trajectory of 'Physics is ...'")
xlabel!("x-dimension")
ylabel!("y-dimension")

#+attr_org: :width 1146

In [1]:
print(response_clear_specific_paragraph.response.choices[1]["message"]["content"])

the study of matter, energy, and the interactions between them.

The subfields of physics include mechanics, thermodynamics, electromagnetism, optics, quantum mechanics, and particle physics.

Physics affects everyday life in numerous ways. It explains how objects move, why the sky is blue, and how electricity powers our devices. It also helps develop new technologies such as medical imaging, renewable energy sources, and communication systems. Understanding physics allows us to make informed decisions and advancements in various fields, from transportation and engineering to medicine and environmental conservation.